This notebook will utilize the MNIST dataset to aid in training a neural network (NN) that can recognize digits. Its performance will then be test on some Sudoku puzzles, but it is expected as of now that it may not be perfect since this dataset containts handwritten digits. Sudoku puzzles are typically printed in a nicer fashion. Therefore, it may be wise to explore and expand the training dataset to also incorporate printed Sudoku puzzles. This may make it easier for the NN to then recognize puzzles that have been printed and partially filled in. 

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch.utils.data import random_split, TensorDataset
import cv2
from copy import copy
import re
import os

from python_helper_functions import parse_dat, get_sudoku_dataset
from NumberModel import NumberModel, fit

In [2]:
print(parse_dat('./warped_dataset/image1082.dat'))

[[3 4 7 0 5 1 0 0 0]
 [0 1 9 0 0 0 0 0 0]
 [5 0 6 3 4 0 0 0 0]
 [0 7 0 0 0 3 0 0 2]
 [0 0 1 0 0 0 7 0 0]
 [9 0 0 8 0 0 0 5 0]
 [0 0 0 0 3 5 6 0 8]
 [0 0 0 0 0 0 3 2 0]
 [0 0 0 6 8 0 5 9 1]]


In [8]:
warped_path = "./warped_dataset/"
cell_path = "./cell_dataset/"
all_filenames = os.listdir(warped_path)
warped_jpg_filenames = sorted([filename for filename in all_filenames if re.search(r"\.jpg", filename)])
warped_dat_filenames = sorted([filename for filename in all_filenames if re.search(r"\.dat", filename)])

folder_counter = np.zeros(10, dtype=np.int32)
for i in range(len(warped_dat_filenames)):
    curr_dat = warped_path + warped_dat_filenames[i]
    curr_jpg = warped_path + warped_jpg_filenames[i]
    full_img = cv2.imread(curr_jpg, cv2.IMREAD_GRAYSCALE)

    cells = []
    step_x = full_img.shape[1] // 9
    step_y = full_img.shape[0] // 9
    for m in range(9):
        for n in range(9):
            curr_cell = copy(full_img[m*step_y: (m+1)*step_y, n*step_x: (n+1)*step_x])
            cells.append(curr_cell)

    labels = parse_dat(curr_dat).flatten()

    for k, label in enumerate(labels):
        label_path = cell_path + str(label) + "/"
        if not os.path.exists(label_path):
            os.makedirs(label_path)

        sub_image = cells[k]
        cv2.imwrite(label_path + str(folder_counter[label]) + ".jpg", sub_image)
        folder_counter[label] += 1

In [2]:
train_data, test_data, single_image_dimension = get_sudoku_dataset()
losses, NetObject = fit(train_data, test_data, single_image_dimension, 20, 50)

Shape labels: torch.Size([11502]), dtype: torch.int32
Shape all_images: torch.Size([11502, 1, 50, 50]), dtype: torch.float32
Epoch 0

TypeError: only integer tensors of a single element can be converted to an index